In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [325]:
vod_08 = pd.read_csv('../data/데이터스쿨3차_2308월/데이터스쿨_3차_VOD_2308.csv', encoding = 'cp949', sep = '\t')
vod_09 = pd.read_csv('../data/데이터스쿨3차_2309월/데이터스쿨_3차_VOD_2309.csv', encoding = 'cp949', sep = '\t')

vod_08_aug1 = vod_08.copy()
vod_09_aug1 = vod_09.copy()

vod_08_aug1.subsr = vod_08_aug1.subsr * 10
vod_09_aug1.subsr = vod_09_aug1.subsr * 10

In [358]:
vod_08_aug2 = vod_08.copy()
vod_09_aug2 = vod_09.copy()

vod_08_aug2.subsr = vod_08_aug2.subsr * 100
vod_09_aug2.subsr = vod_09_aug2.subsr * 100

In [366]:
# # 8,9월 데이터 합치기
# vod_89 = pd.concat([vod_08, vod_08_aug1, vod_08_aug2, vod_09, vod_09_aug1, vod_09_aug2], ignore_index=True)
# vod_89
vod_89 = pd.concat([vod_08, vod_09], ignore_index=True)
vod_89

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,65941000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230812163507
1,66873000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230816205227
2,66873000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,718,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230829194727
3,61689000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230813130609
4,61619000,꼬리에꼬리를무는그날이야기 37회(22/07/21),TV 시사/교양,기타,4183,"살인범의 미토콘드리아 - 2006 냉동고 살인사건. 2006년 7월 23일, 서울 ...","장도연,장현성,장성규",1:13,20230804092737
...,...,...,...,...,...,...,...,...,...
10652,60169000,간질간질,키즈,학습,360,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...,-,0:06,20230914211110
10653,66317000,알사탕,키즈,학습,480,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고...",-,0:08,20230903223544
10654,66317000,알사탕,키즈,학습,200,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고...",-,0:08,20230903223210
10655,66900000,월간 아기상어(영어) 01회,키즈,학습,46,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...,아기상어,0:01,20230904120057


In [367]:
def preprocessing(data):
  df = data.copy()
  
  # disp_rtm 문자열을 분 단위로 변경
  def convert_runtime(runtime_str):
    # 입력값이 NaN이면 0 반환
    if pd.isna(runtime_str):
        return 0
    hours = int(runtime_str.split(':')[0])
    minutes = int(runtime_str.split(':')[1])
    total_minutes = hours * 60 + minutes
    return total_minutes

  df['disp_rtm'] = df['disp_rtm'].apply(convert_runtime)

  # 런타임 0분인 것 제거
  df = df[df['disp_rtm'] != 0]

  # 무삭제 제거
  df['asset_nm'] = df['asset_nm'].str.replace(r'무삭제판|무삭제', '', regex = True)
  
  # 예고편 제거
  df = df[~df['asset_nm'].str.contains(r'\(예고편\)|\(예고\)', regex=True)]
  
  # 예약구매, 사전구매 제거
  df = df[~df['asset_nm'].str.contains('예약구매|사전구매')]

  # 00회가 포함된 행은 런타임이 0또는 1이므로 제거
  df = df[~df['asset_nm'].str.contains(r'\b00회\b')]
  
  ## 괄호와 그 안의 내용 제거 
  df['asset_nm'] = df['asset_nm'].str.replace(r'\([^()]*\)', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\.\.\.', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\…', '', regex=True) # … 기호 제거
  df['asset_nm'] = df['asset_nm'].str.replace('-', " ")
  df['asset_nm'] = df['asset_nm'].str.rstrip('.')
  # df['asset_nm'] = df['asset_nm'].str.strip()

  # df3['series_nm'] = df3['asset_nm'].str.replace(r'\d+회$', '', regex=True)
  # df3['series_nm'] = df3['series_nm'].str.replace(r'\d+회\.', '', regex=True) 
  # df3['series_nm'] = df3['series_nm'].str.replace(r'\d+화$', regex = True)

  # use_tms 분 단위로 변경
  df['use_tms'] = round(df['use_tms'] / 60, 1)

  # 셋탑번호가 66056000인 것 삭제 - content 에서 이상치 아이디로 판별
  df = df[df['subsr'] != 66056000]
  df = df[df['subsr'] != 660560000]
  df = df[df['subsr'] != 6605600000]

  return df

In [368]:
df = preprocessing(vod_89)
df.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,65941000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230812163507
1,66873000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230816205227
2,66873000,그것이알고싶다 1361회,TV 시사/교양,기타,12.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230829194727
3,61689000,그것이알고싶다 1361회,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230813130609
4,61619000,꼬리에꼬리를무는그날이야기 37회,TV 시사/교양,기타,69.7,"살인범의 미토콘드리아 - 2006 냉동고 살인사건. 2006년 7월 23일, 서울 ...","장도연,장현성,장성규",73,20230804092737


In [369]:
# 프로그램 회차별 정보
df1 = df[['asset_nm', 'ct_cl', 'genre_of_ct_cl', 'ACTR_DISP', 'disp_rtm']].drop_duplicates().reset_index(drop = True)
df1.head()

,asset_nm,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm
0,그것이알고싶다 1361회,TV 시사/교양,기타,김상중,80
1,꼬리에꼬리를무는그날이야기 37회,TV 시사/교양,기타,"장도연,장현성,장성규",73
2,꼬리에꼬리를무는그날이야기 61회,TV 시사/교양,기타,"장도연,장현성,장성규",73
3,인간극장 3338회,TV 시사/교양,기타,명선 스님,32
4,꼬리에꼬리를무는그날이야기 89회,TV 시사/교양,기타,"장도연,장현성,장성규",78


In [370]:
# 유저마다 각 회차를 본 시간을 더함
df2 = pd.DataFrame(df.groupby(['subsr','asset_nm', 'ct_cl', 'genre_of_ct_cl', 'disp_rtm'])['use_tms'].sum()).reset_index()
df2 = df2[(df2['disp_rtm'] != 0) & (df2['use_tms'] != 0)]
df2

,subsr,asset_nm,ct_cl,genre_of_ct_cl,disp_rtm,use_tms
0,59879000,소방서 옆 경찰서 05회,TV드라마,기타,69,2.4
1,59879000,소방서 옆 경찰서 06회,TV드라마,기타,59,22.1
2,59879000,신성한 이혼 01회,TV드라마,기타,64,12.6
3,59879000,신성한 이혼 02회,TV드라마,기타,62,16.8
4,59879000,신성한 이혼 03회,TV드라마,기타,63,4.3
...,...,...,...,...,...,...
5705,67148000,타요의 씽씽극장 동요2 10회,키즈,기타,2,2.0
5706,67148000,타요의 씽씽극장 동요2 11회,키즈,기타,2,0.5
5707,67154000,스트릿 우먼 파이터 2 04회,TV 연예/오락,기타,131,1.2
5708,67161000,스파이 코드명 포춘,영화,액션/어드벤쳐,114,0.2


In [430]:
# 시청시간으로 유저가 이 회차를 시청했는지 여부를 결정
## 시청시간을 런타임으로 나눠 0.1 이 넘으면 시청한 것으로 간주
df2['watched'] = df2['use_tms'] / df2['disp_rtm']
df2['watched'] = df2['watched'].apply(lambda x : 1 if x >= 0.5 else 0)
df2.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,disp_rtm,use_tms,watched,series_nm
0,59879000,소방서 옆 경찰서 05회,TV드라마,기타,69,2.4,0,소방서 옆 경찰서
1,59879000,소방서 옆 경찰서 06회,TV드라마,기타,59,22.1,0,소방서 옆 경찰서
2,59879000,신성한 이혼 01회,TV드라마,기타,64,12.6,0,신성한 이혼
3,59879000,신성한 이혼 02회,TV드라마,기타,62,16.8,0,신성한 이혼
4,59879000,신성한 이혼 03회,TV드라마,기타,63,4.3,0,신성한 이혼


In [431]:
# 회차 제거
df11 = df1.copy()

df11['asset_nm'] = df11['asset_nm'].str.replace(r'\d+회$', '', regex=True)
df11['asset_nm'] = df11['asset_nm'].str.replace(r'\d+회\.', '', regex=True)
df11['asset_nm'] = df11['asset_nm'].str.replace(r'\d+화$', '', regex=True)
df11['asset_nm'] = df11['asset_nm'].str.strip()
df11['asset_nm'] = df11['asset_nm'].str.rstrip('.')
df11

,asset_nm,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm
0,그것이알고싶다,TV 시사/교양,기타,김상중,80
1,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,"장도연,장현성,장성규",73
2,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,"장도연,장현성,장성규",73
3,인간극장,TV 시사/교양,기타,명선 스님,32
4,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,"장도연,장현성,장성규",78
...,...,...,...,...,...
3850,엄마가 화났다,키즈,학습,-,8
3851,핑크퐁 자동차 동화,키즈,학습,핑크퐁,4
3852,간질간질,키즈,학습,-,6
3853,월간 아기상어,키즈,학습,아기상어,1


In [432]:
# 유저 시청 정보의 회차 제거한 변수 추가
df2['series_nm'] = df2['asset_nm'].str.replace(r'\d+회$', '', regex=True)
df2['series_nm'] = df2['series_nm'].str.replace(r'\d+회\.', '', regex=True)
df2['series_nm'] = df2['series_nm'].str.replace(r'\d+화$', '', regex=True)
df2['series_nm'] = df2['series_nm'].str.strip()
df2['series_nm'] = df2['series_nm'].str.rstrip('.')
df2.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,disp_rtm,use_tms,watched,series_nm
0,59879000,소방서 옆 경찰서 05회,TV드라마,기타,69,2.4,0,소방서 옆 경찰서
1,59879000,소방서 옆 경찰서 06회,TV드라마,기타,59,22.1,0,소방서 옆 경찰서
2,59879000,신성한 이혼 01회,TV드라마,기타,64,12.6,0,신성한 이혼
3,59879000,신성한 이혼 02회,TV드라마,기타,62,16.8,0,신성한 이혼
4,59879000,신성한 이혼 03회,TV드라마,기타,63,4.3,0,신성한 이혼


In [433]:
df3 = df2[['subsr', 'series_nm', 'ct_cl',	'genre_of_ct_cl', 'watched']].copy()
df3.head()

,subsr,series_nm,ct_cl,genre_of_ct_cl,watched
0,59879000,소방서 옆 경찰서,TV드라마,기타,0
1,59879000,소방서 옆 경찰서,TV드라마,기타,0
2,59879000,신성한 이혼,TV드라마,기타,0
3,59879000,신성한 이혼,TV드라마,기타,0
4,59879000,신성한 이혼,TV드라마,기타,0


In [434]:
# 유저의 시리즈별 시청 횟수
df4 = df3.groupby(['subsr', 'series_nm', 'ct_cl', 'genre_of_ct_cl'])['watched'].sum().reset_index()
df4 = df4[df4['watched']!= 0]
df4

,subsr,series_nm,ct_cl,genre_of_ct_cl,watched
2,59895000,금이야 옥이야,TV드라마,기타,1
3,59900000,2022 역사저널 그날,TV 시사/교양,기타,1
4,59900000,그것이알고싶다,TV 시사/교양,기타,3
8,59900000,범죄도시3,영화,액션/어드벤쳐,1
16,59930000,가면의 여왕,TV드라마,기타,6
...,...,...,...,...,...
2066,67140000,경남 통영 2부,우리동네,연예/오락,1
2067,67140000,밀수,영화,액션/어드벤쳐,1
2068,67140000,잠자는 숲속의 공주,키즈,기타,1
2070,67148000,타요의 씽씽극장 동요2,키즈,기타,10


In [435]:
# 8~9월 시리즈별 시청된 총 횟수
df5 = pd.DataFrame(df11[['asset_nm', 'ct_cl', 'genre_of_ct_cl']].value_counts().reset_index())
df5.columns = ['series_nm', 'ct_cl', 'genre_of_ct_cl', 'watched_all']
df5

,series_nm,ct_cl,genre_of_ct_cl,watched_all
0,금이야 옥이야,TV드라마,기타,83
1,연희공략: 건륭황제의여인,TV드라마,외화 시리즈,63
2,TV소설 은희,TV드라마,기타,63
3,인간극장,TV 시사/교양,기타,55
4,런닝맨,TV 연예/오락,기타,54
...,...,...,...,...
1019,봉신연의,영화,액션/어드벤쳐,1
1020,부산,영화,액션/어드벤쳐,1
1021,북 오브 러브,영화,멜로,1
1022,분노의 질주: 라이드 오어 다이,영화,액션/어드벤쳐,1


In [436]:
df6 = df4.merge(df5, on = ['series_nm', 'ct_cl', 'genre_of_ct_cl'], how = 'left')
df6

,subsr,series_nm,ct_cl,genre_of_ct_cl,watched,watched_all
0,59895000,금이야 옥이야,TV드라마,기타,1,83
1,59900000,2022 역사저널 그날,TV 시사/교양,기타,1,3
2,59900000,그것이알고싶다,TV 시사/교양,기타,3,21
3,59900000,범죄도시3,영화,액션/어드벤쳐,1,1
4,59930000,가면의 여왕,TV드라마,기타,6,6
...,...,...,...,...,...,...
1249,67140000,경남 통영 2부,우리동네,연예/오락,1,1
1250,67140000,밀수,영화,액션/어드벤쳐,1,2
1251,67140000,잠자는 숲속의 공주,키즈,기타,1,1
1252,67148000,타요의 씽씽극장 동요2,키즈,기타,10,13


In [437]:
# 유저의 프로그램 총 시청 횟수
df7 = df6.groupby(['subsr'])['watched'].sum().reset_index()
df7.columns = ['subsr', 'watched_cnt']
df7

,subsr,watched_cnt
0,59895000,1
1,59900000,5
2,59930000,11
3,59933000,9
4,60040000,1
...,...,...
294,67107000,2
295,67117000,10
296,67140000,3
297,67148000,10


In [438]:
df8 = df6.merge(df7, on = 'subsr', how = 'left')
df8 = df8[df8['watched_cnt'] != 0].reset_index(drop = True)
df8

,subsr,series_nm,ct_cl,genre_of_ct_cl,watched,watched_all,watched_cnt
0,59895000,금이야 옥이야,TV드라마,기타,1,83,1
1,59900000,2022 역사저널 그날,TV 시사/교양,기타,1,3,5
2,59900000,그것이알고싶다,TV 시사/교양,기타,3,21,5
3,59900000,범죄도시3,영화,액션/어드벤쳐,1,1,5
4,59930000,가면의 여왕,TV드라마,기타,6,6,11
...,...,...,...,...,...,...,...
1249,67140000,경남 통영 2부,우리동네,연예/오락,1,1,3
1250,67140000,밀수,영화,액션/어드벤쳐,1,2,3
1251,67140000,잠자는 숲속의 공주,키즈,기타,1,1,3
1252,67148000,타요의 씽씽극장 동요2,키즈,기타,10,13,10


In [439]:
# 시청횟수가 3개 이상인 유저만 
user_cnt = df8['subsr'].value_counts()
filter_users = user_cnt[user_cnt >= 3].index
vod_over3 = df8[df8['subsr'].isin(filter_users)]
vod_over3

,subsr,series_nm,ct_cl,genre_of_ct_cl,watched,watched_all,watched_cnt
1,59900000,2022 역사저널 그날,TV 시사/교양,기타,1,3,5
2,59900000,그것이알고싶다,TV 시사/교양,기타,3,21,5
3,59900000,범죄도시3,영화,액션/어드벤쳐,1,1,5
4,59930000,가면의 여왕,TV드라마,기타,6,6,11
5,59930000,귀신이산다,영화,코미디,1,1,11
...,...,...,...,...,...,...,...
1240,67055000,실화탐사대,TV 시사/교양,기타,2,9,80
1241,67055000,오은영 리포트 결혼 지옥,TV 시사/교양,기타,2,23,80
1249,67140000,경남 통영 2부,우리동네,연예/오락,1,1,3
1250,67140000,밀수,영화,액션/어드벤쳐,1,2,3


In [440]:
# 시청횟수가 1개 이상인 유저(모든 유저) 
user_cnt = df8['subsr'].value_counts()
filter_users = user_cnt[user_cnt >= 1].index
vod_all = df8[df8['subsr'].isin(filter_users)]
vod_all

,subsr,series_nm,ct_cl,genre_of_ct_cl,watched,watched_all,watched_cnt
0,59895000,금이야 옥이야,TV드라마,기타,1,83,1
1,59900000,2022 역사저널 그날,TV 시사/교양,기타,1,3,5
2,59900000,그것이알고싶다,TV 시사/교양,기타,3,21,5
3,59900000,범죄도시3,영화,액션/어드벤쳐,1,1,5
4,59930000,가면의 여왕,TV드라마,기타,6,6,11
...,...,...,...,...,...,...,...
1249,67140000,경남 통영 2부,우리동네,연예/오락,1,1,3
1250,67140000,밀수,영화,액션/어드벤쳐,1,2,3
1251,67140000,잠자는 숲속의 공주,키즈,기타,1,1,3
1252,67148000,타요의 씽씽극장 동요2,키즈,기타,10,13,10


In [441]:
# 시청 여부, 해당 시리즈 총 횟수, 유저의 프로그램 총 시청 횟수를 이용해
# 유저의 프로그램 선호도를 측정
import numpy as np
def scoring(df):
  N = df['watched_all'] # 해당 프로그램 전체 회차 수
  L = df['watched_cnt'] # 유저의 총 시청 프로그램 회차 수
  n = df['watched'] # 유저의 해당 프로그램 시청 수
  lam = np.log(2) / 2
  w1 = 1 - np.exp(-1 * lam * N)
  data = pd.DataFrame([N, L]).T
  def custom_weight(data):
    if data['watched_cnt'] < data['watched_all']:
      return data['watched_all'] / data['watched_cnt']
    else:
      return 1

  w2 = data.apply(custom_weight, axis = 1)

  score = (n / N) * w1 * w2

  return score

In [442]:
vod_over3['score'] = scoring(vod_over3) 
vod_all['score'] = scoring(vod_all)

In [443]:
vod_over3 = vod_over3[['subsr', 'series_nm', 'ct_cl', 'genre_of_ct_cl', 'score']]
vod_over3 = vod_over3.sort_values(by = 'subsr').reset_index(drop = True)
vod_over3['category'] = vod_over3['ct_cl'].apply(lambda x : x if x in ['영화', '키즈'] else 'TV프로그램')
vod_over3['rename'] = vod_over3['series_nm'].apply(lambda x : x.replace(' ', '') if isinstance(x, str) else x)
vod_over3

,subsr,series_nm,ct_cl,genre_of_ct_cl,score,category,rename
0,59900000,2022 역사저널 그날,TV 시사/교양,기타,0.215482,TV프로그램,2022역사저널그날
1,59900000,그것이알고싶다,TV 시사/교양,기타,0.599586,TV프로그램,그것이알고싶다
2,59900000,범죄도시3,영화,액션/어드벤쳐,0.292893,영화,범죄도시3
3,59930000,가면의 여왕,TV드라마,기타,0.875000,TV프로그램,가면의여왕
4,59930000,귀신이산다,영화,코미디,0.292893,영화,귀신이산다
...,...,...,...,...,...,...,...
1002,67055000,신발 벗고 돌싱포맨,TV 연예/오락,기타,0.096875,TV프로그램,신발벗고돌싱포맨
1003,67055000,오은영 리포트 결혼 지옥,TV 시사/교양,기타,0.086926,TV프로그램,오은영리포트결혼지옥
1004,67140000,밀수,영화,액션/어드벤쳐,0.250000,영화,밀수
1005,67140000,경남 통영 2부,우리동네,연예/오락,0.292893,TV프로그램,경남통영2부


In [444]:
vod_all = vod_all[['subsr', 'series_nm', 'ct_cl', 'genre_of_ct_cl', 'score']]
vod_all = vod_all.sort_values(by = 'subsr').reset_index(drop = True)
vod_all['category'] = vod_all['ct_cl'].apply(lambda x : x if x in ['영화', '키즈'] else 'TV프로그램')
vod_all['rename'] = vod_all['series_nm'].apply(lambda x : x.replace(' ', '') if isinstance(x, str) else x)
vod_all

,subsr,series_nm,ct_cl,genre_of_ct_cl,score,category,rename
0,59895000,금이야 옥이야,TV드라마,기타,1.000000,TV프로그램,금이야옥이야
1,59900000,2022 역사저널 그날,TV 시사/교양,기타,0.215482,TV프로그램,2022역사저널그날
2,59900000,그것이알고싶다,TV 시사/교양,기타,0.599586,TV프로그램,그것이알고싶다
3,59900000,범죄도시3,영화,액션/어드벤쳐,0.292893,영화,범죄도시3
4,59930000,가면의 여왕,TV드라마,기타,0.875000,TV프로그램,가면의여왕
...,...,...,...,...,...,...,...
1249,67140000,경남 통영 2부,우리동네,연예/오락,0.292893,TV프로그램,경남통영2부
1250,67140000,밀수,영화,액션/어드벤쳐,0.250000,영화,밀수
1251,67140000,잠자는 숲속의 공주,키즈,기타,0.292893,키즈,잠자는숲속의공주
1252,67148000,타요의 씽씽극장 동요2,키즈,기타,0.988951,키즈,타요의씽씽극장동요2


In [456]:
vod = vod_all.copy()

In [457]:
vod_id = pd.read_csv('../data/vod_list_add10.csv', index_col=0)
vod_id = vod_id[['rename', 'Category', 'vod_id']]
vod_id.columns = ['rename', 'category', 'vod_id']
vod_id

,rename,category,vod_id
0,와이낫크루뜻밖의여행,TV프로그램,0
1,그레이트뷰티,영화,1
2,해안선,영화,2
3,아는형님,TV프로그램,3
4,베놈,영화,4
...,...,...,...
5019,일루셔니스트,영화,5019
5020,"파리,13구",영화,5020
5021,미쓰백,영화,5021
5022,그녀는거짓말을너무사랑해,영화,5022


In [458]:
vod1 = vod.merge(vod_id, on = ['rename', 'category'], how = 'left')
vod_score = vod1[['subsr', 'vod_id', 'score']]
vod_score

,subsr,vod_id,score
0,59895000,784,1.000000
1,59900000,572,0.215482
2,59900000,297,0.599586
3,59900000,2539,0.292893
4,59930000,728,0.875000
...,...,...,...
1249,67140000,4398,0.292893
1250,67140000,2867,0.250000
1251,67140000,689,0.292893
1252,67148000,880,0.988951


In [459]:
# 필요한 Surprise 알고리즘 불러오기
from surprise import SVD, BaselineOnly, SVDpp, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import KNNBaseline, NMF, KNNWithMeans, KNNBasic
from surprise import accuracy
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# surprise 데이터 형식으로 변환
def convert_traintest_dataframe_forsurprise(training_dataframe):
    reader = Reader(rating_scale=(0, 1)) # 이 범위를 넘으면 양극값으로 대체
    trainset = Dataset.load_from_df(training_dataframe[['subsr', 'vod_id', 'score']], reader)
    trainset = trainset.construct_trainset(trainset.raw_ratings)
    return trainset

trainset = convert_traintest_dataframe_forsurprise(vod_score)

In [460]:
sim_options = {'name': 'pearson_baseline', 'user_based': False} # item-based similarity
bsl_options = {'method' : 'sgd', 'n_epochs' : 1}
knnbaseline = KNNBaseline(k = 40, sim_options=sim_options, random_state = 42, min_k= 1, 
                          bsl_options=bsl_options)

knnbaseline.fit(trainset)

Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [461]:
user_id = sorted(vod_score.subsr.unique())
vod_id = sorted(vod_score.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(knnbaseline.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59895000,3,0.397710
1,59895000,6,0.396788
2,59895000,7,0.394305
3,59895000,8,0.403292
4,59895000,10,0.397296
...,...,...,...
217069,67164000,4629,0.394818
217070,67164000,4647,0.394815
217071,67164000,4685,0.394803
217072,67164000,4698,0.394819


In [462]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:25].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
0,59895000,747,738,784,296,143,1166,880,1156,888,...,111,818,41,554,405,14,1077,398,884,394
1,59900000,296,143,8,111,16,292,256,531,3028,...,44,297,1166,880,1156,888,1234,848,401,1347
2,59930000,728,456,296,143,1166,880,1156,888,1234,...,818,41,554,405,14,1077,398,884,394,720
3,59933000,296,143,8,292,256,36,531,48,861,...,13,432,44,297,1166,880,1156,888,1234,848
4,60040000,296,143,1166,880,1156,888,1234,848,8,...,554,405,14,1077,398,884,394,720,954,1086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,67107000,296,143,1166,880,1156,888,1234,848,8,...,554,405,14,1077,398,884,394,720,954,1086
295,67117000,818,296,143,1166,880,1156,888,1234,848,...,554,405,14,1077,398,884,394,720,954,1086
296,67140000,296,143,1166,880,1156,888,1234,848,8,...,554,405,14,1077,398,884,394,720,954,1086
297,67148000,880,296,143,1166,1156,888,1234,848,8,...,554,405,14,1077,398,884,394,720,954,1086


In [463]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59895000,"[747, 738, 784, 296, 143, 1166, 880, 1156, 888..."
1,59900000,"[296, 143, 8, 111, 16, 292, 256, 531, 3028, 24..."
2,59930000,"[728, 456, 296, 143, 1166, 880, 1156, 888, 123..."
3,59933000,"[296, 143, 8, 292, 256, 36, 531, 48, 861, 3028..."
4,60040000,"[296, 143, 1166, 880, 1156, 888, 1234, 848, 8,..."
...,...,...
294,67107000,"[296, 143, 1166, 880, 1156, 888, 1234, 848, 8,..."
295,67117000,"[818, 296, 143, 1166, 880, 1156, 888, 1234, 84..."
296,67140000,"[296, 143, 1166, 880, 1156, 888, 1234, 848, 8,..."
297,67148000,"[880, 296, 143, 1166, 1156, 888, 1234, 848, 8,..."


In [464]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
vod_list = pd.read_csv('../data/vod_list_add10.csv', index_col=0)
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = vod_score[vod_score.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue
    
    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
vod_predict_1

,subsr,vod_id
0,59895000,"[747.0, 738.0, 784.0, 296.0, 143.0, 1166.0, 88..."
1,59900000,"[296.0, 143.0, 8.0, 111.0, 16.0, 292.0, 256.0,..."
2,59930000,"[728.0, 456.0, 296.0, 143.0, 1166.0, 880.0, 11..."
3,59933000,"[296.0, 143.0, 8.0, 292.0, 256.0, 36.0, 531.0,..."
4,60040000,"[296.0, 143.0, 1166.0, 880.0, 1156.0, 888.0, 1..."
...,...,...
294,67107000,"[296.0, 143.0, 1166.0, 880.0, 1156.0, 888.0, 1..."
295,67117000,"[818.0, 296.0, 143.0, 1166.0, 880.0, 1156.0, 8..."
296,67140000,"[296.0, 143.0, 1166.0, 880.0, 1156.0, 888.0, 1..."
297,67148000,"[880.0, 296.0, 143.0, 1166.0, 1156.0, 888.0, 1..."


In [465]:
testdata = pd.read_csv('../data/watched_vod_10.csv', index_col=0)
testdata = testdata.groupby('subsr')['vod_id'].unique().reset_index()
testdata

,subsr,vod_id
0,59900000,"[1278, 1885]"
1,59930000,[2098]
2,59933000,[296]
3,60050000,"[200, 2415, 2869, 2546, 213, 2291, 2610, 4880]"
4,60067000,"[1785, 887, 1347, 4773, 4782, 4759, 992, 3806,..."
...,...,...
215,67133000,[296]
216,67164000,"[4956, 1434, 4995, 2058]"
217,67170000,"[3464, 948, 3900, 675]"
218,67202000,[1028]


In [466]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp
        
    return precision_value / merge_df.shape[0]

precision_k(testdata, vod_predict_1)

0.06217948717948713

In [ ]:
# 증강 over3 0.07
# 증강 under3